In [1]:
import pandas as pd

1. Write a function that estimates the emission parameters from the training set using MLE (maximum likelihoodestimation):
Which training set?

file_to_df is a function to convert some data file into a dataframe.

In [2]:
def file_to_df (file_path):


  with open(file_path, 'r',encoding='utf-8') as file:
      lines = file.readlines()
      data = [line.strip().split(' ', maxsplit=2)[:2] for line in lines]

  # Create the dataframe
  df = pd.DataFrame(data, columns=['x', 'y'])
  #drop rows where value for y=None
  df = df.dropna(subset=['y'])

  df['x'] = df['x'].astype(str)
  df['y'] = df['y'].astype(str)
  # Display the dataframe
  #print("Here is dataframe created for the file path: ",file_path,"\n")
  #print(df.to_string)
  return df

file_path_train_es='Data/ES/train'
df_train_es=file_to_df(file_path_train_es)

In [3]:
def count_y(df,y_value):
    unique_counts = df['y'].value_counts().to_dict()
    print("unique counts:",unique_counts)
    return unique_counts[y_value]


In [4]:
def y_star_with_smallest_count(df):
     unique_counts = df['y_star'].value_counts().to_dict()
     min_count = min(unique_counts.values())
     y_with_min_count = [key for key, value in unique_counts.items() if value == min_count]         
     return y_with_min_count[0]         

In [5]:
def create_df_filtered_for_y_value(df,y_value):
  return df[df['y'] == y_value]


In [6]:
def create_df_x_count_y_to_x(df):
    df_x_count_y_to_x = df['x'].value_counts().reset_index()

    df_x_count_y_to_x.columns = ['x', 'count_y_to_x']

    return df_x_count_y_to_x

In [7]:
def create_ls_of_all_y_values(df):
    unique_values = df['y'].unique()
    return (unique_values)
print(create_ls_of_all_y_values(df_train_es))

['O' 'B-positive' 'B-negative' 'B-neutral' 'I-neutral' 'I-positive'
 'I-negative']


In [8]:
def create_df_e_x_y_train(train_df,y_value):
   
    df_train_filtered_for_y = create_df_filtered_for_y_value(train_df,y_value)
    df_e_x_y_train=create_df_x_count_y_to_x(df_train_filtered_for_y)
    df_e_x_y_train['e(x|y)'] = df_e_x_y_train['count_y_to_x']/(count_y(train_df,y_value))
    df_e_x_y_train['y']=y_value
    return df_e_x_y_train


df_e_x_y_train_for_I_neutral=create_df_e_x_y_train(df_train_es,"B-positive")
print(df_e_x_y_train_for_I_neutral)
print(df_e_x_y_train_for_I_neutral['e(x|y)'].sum())

unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
               x  count_y_to_x    e(x|y)           y
0         comida           169  0.145690  B-positive
1       servicio           122  0.105172  B-positive
2    restaurante            46  0.039655  B-positive
3          trato            44  0.037931  B-positive
4       ambiente            33  0.028448  B-positive
..           ...           ...       ...         ...
280       dorada             1  0.000862  B-positive
281      detalle             1  0.000862  B-positive
282   cantidades             1  0.000862  B-positive
283       Atteca             1  0.000862  B-positive
284        menus             1  0.000862  B-positive

[285 rows x 4 columns]
1.0000000000000002


In [9]:
def create_df_e_x_y_test(test_df,y_value,train_df):
    k=1
    #print(test_df.info())
    df_test_filtered_for_y = create_df_filtered_for_y_value(test_df,y_value)
    #print(df_test_filtered_for_y)
    df_e_x_y_test=create_df_x_count_y_to_x(df_test_filtered_for_y)
    #print (df_e_x_y_test)
    
    count_y_plus_k=count_y(test_df,y_value)+k

    #make a list of all the x values in the train_df's x column
   

    
    train_df_x_values = train_df['x'].tolist()
    #print(train_df_x_values)
    
    df_e_x_y_test['x'] = df_e_x_y_test['x'].apply(lambda x: x if x in train_df_x_values else "#UNK#") #if word does or does not appear in training set
    #print(df_e_x_y_test)
    
    df_e_x_y_test['e(x|y)'] = df_e_x_y_test.apply(lambda row: (row['count_y_to_x'] / count_y_plus_k) if row['x'] != '#UNK#' else (k / count_y_plus_k), axis=1)
    print(df_e_x_y_test['e(x|y)'].sum())

    #case 1
    #df_e_x_y_test['e(x|y)'] = df_e_x_y_test['count_y_to_x']/count_y_plus_k

    #case 2
    #df_e_x_y_test['e(x|y)'] = k/count_y_plus_k


    return df_e_x_y_test


file_path_test_es='Data/ES/dev.out'
df_test_es=file_to_df(file_path_test_es)
df_e_x_y_test_for_I_neutral=create_df_e_x_y_test(df_test_es,"B-positive",df_train_es)
print(df_e_x_y_test_for_I_neutral)
# count_unk = 0
# value_counts = df_e_x_y_test_for_I_neutral['x'].value_counts()
# print(value_counts)
# if '#UNK#' in value_counts.index:
#     count_unk = value_counts['#UNK#']
# print(count_unk)

unique counts: {'O': 3998, 'B-positive': 160, 'B-negative': 61, 'I-positive': 49, 'I-negative': 36, 'B-neutral': 8}
0.9937888198757765
               x  count_y_to_x    e(x|y)
0       servicio            24  0.149068
1         comida            23  0.142857
2    restaurante             8  0.049689
3       ambiente             7  0.043478
4         platos             6  0.037267
..           ...           ...       ...
67      cocinero             1  0.006211
68        tartar             1  0.006211
69       postres             1  0.006211
70  localización             1  0.006211
71         comer             1  0.006211

[72 rows x 3 columns]


In [37]:


def create_e_x_y_df_train_all_y_values(file_path):
    
  df_train=file_to_df(file_path)
  #print(df_train_es)
  ls_df_train=[]
  ls_y_values=create_ls_of_all_y_values(df_train)
  #print(len(ls_y_values))
  for y_value in ls_y_values:
    if y_value!=None:
      df_e_x_y=create_df_e_x_y_train(df_train,y_value)
      ls_df_train.append(df_e_x_y)
  #print(len(ls_df_train))
  
  combined_df_train=pd.concat(ls_df_train,axis=0)
  return (combined_df_train)
#file_path_train_es='Data/ES/train'
#print(create_e_x_y_df_train_all_y_values(file_path_train_es))
display(create_e_x_y_df_train_all_y_values(file_path_train_es))

unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}


,x,count_y_to_x,e(x|y),y
0,",",1664,0.057310,O
1,.,1623,0.055898,O
2,y,1024,0.035268,O
3,de,1011,0.034820,O
4,que,845,0.029103,O
...,...,...,...,...
108,está,1,0.005848,I-negative
109,sopa,1,0.005848,I-negative
110,tabaco,1,0.005848,I-negative
111,gambas,1,0.005848,I-negative


In [36]:
def create_df_x_to_y_star(file_path):
    e_x_y_df_train=create_e_x_y_df_train_all_y_values(file_path)
    
    # Group by 'x' and find the maximum 'e(x|y)' value for each group
    df_x_to_y_star = e_x_y_df_train.groupby('x')['e(x|y)'].max().reset_index()

    # Find the corresponding 'y' values for the maximum 'e(x|y)' values
    df_x_to_y_star = pd.merge(df_x_to_y_star, e_x_y_df_train, on=['x', 'e(x|y)'])

    # Rename the columns
    df_x_to_y_star.columns = ['x', 'max_e(x|y)','count_y_to_x', 'y_star']

    # Print the resulting DataFrame
    return (df_x_to_y_star)
    
file_path_train_es='Data/ES/train'    
df_x_to_y_star=create_df_x_to_y_star(file_path_train_es)
csv_x_to_y_star = df_x_to_y_star.to_csv('train_x_to_y_star.csv', index = True)

unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}


,x,max_e(x|y),count_y_to_x,y_star
0,!,0.005442,158,O
1,"""",0.011696,2,I-negative
2,%,0.000413,12,O
3,(,0.004512,131,O
4,),0.004512,131,O
...,...,...,...,...
5035,“,0.003185,1,I-positive
5036,”,0.003185,1,I-positive
5037,…,0.002625,1,B-negative
5038,″,0.000034,1,O


In [12]:
def generate_y_values(file_path_dev_in,file_path_train,file_path_dev_p1_out):
    df_train=create_df_x_to_y_star(file_path_train)
    x_values=df_train['x'].tolist()
    #the y_label that has the smallest count_y will generate the biggest k / count_y_plus_k which will give the maximum e(x|y) which means that y_label is y_star
    #so that y_label is:
    y_label=y_star_with_smallest_count(df_train) 
    with open(file_path_dev_in, 'r',encoding='utf-8') as file:
      lines = file.readlines()
      for l in range(len(lines)):
        line=lines[l].strip()
        if line in x_values:
          possible_y_values=df_train[df_train['x'] == line]['y_star'].tolist()
          lines[l]=line+" "+possible_y_values[0]
         
          if (len(possible_y_values)!=1):
            print ("something wrong: x_vlaues in df_train not unique for some reason,for line: ",l)
        else:
          if (line!=""):
             line=line+" "+y_label
             lines[l]=line
          else:
            lines[l]=""
          
    with open(file_path_dev_p1_out, 'w',encoding='utf-8') as file:
        
        for line in lines:
          file.write(line+"\n")   
         
         

   
file_path_dev_in_es = 'Data/ES/dev.in'   
file_path_train_es='Data/ES/train'  
file_path_dev_p1_out_es='Data/ES/dev.p1.out' 
generate_y_values(file_path_dev_in_es,file_path_train_es,file_path_dev_p1_out_es)



unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}


In [13]:
file_path_dev_p1_out_es='Data/ES/dev.p1.out'
file_dev_out_es='Data/ES/dev.out'
df_dev_p1_out_es= file_to_df (file_path_dev_p1_out_es)
df_dev_out_es=file_to_df (file_dev_out_es)
print(df_dev_p1_out_es.shape[0])
print(df_dev_out_es.shape[0])

df1=df_dev_p1_out_es
df2=df_dev_out_es

# Compare rows
comparison = df1.equals(df2)

# Get the count of rows that are the same
count_same = df1[df1.eq(df2).all(axis=1)].shape[0]

# Get the count of rows that are not the same
count_not_same = df1[df1.ne(df2).any(axis=1)].shape[0]

# Get the rows that are not the same
rows_not_same = df1[~df1.eq(df2).all(axis=1)]

total_number_of_correctly_predicted_entries=count_same
total_number_of_predicted_entities=df1.shape[0]
precision=total_number_of_correctly_predicted_entries/total_number_of_predicted_entities
print(precision)

# Display the results
# print("Comparison result:", comparison)
# print("Count of rows that are the same:", count_same)
# print("Count of rows that are not the same:", count_not_same)
# print("Rows that are not the same:\n", rows_not_same)

4312
4312
0.6307977736549165


In [16]:
import pandas as pd

# 1. Extracting data
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read().strip().split('\n\n')
    return [sentence.split('\n') for sentence in data]

# Adjusting the paths for ES and RU datasets
train_data = load_data('Data/ES/train')
dev_in_data = load_data('Data/ES/dev.in')
with open('Data/ES/dev.out', 'r', encoding='utf-8') as f:
    dev_tags_actual = [sentence.split() for sentence in f.read().strip().split('\n\n')]

def compute_probabilities(data, state_list):
    start_transition_count = {state: 0 for state in state_list}
    transition_count = {state: {state2: 0 for state2 in state_list} for state in state_list}
    emission_count = {state: {} for state in state_list}
    state_count = {state: 0 for state in state_list}

    for sentence in data:
        prev_state = None
        for line in sentence:
            word, state = line.strip().split()
            if prev_state is None:
                start_transition_count[state] += 1
            else:
                transition_count[prev_state][state] += 1
                emission_count[prev_state][word] = emission_count[prev_state].get(word, 0) + 1
            state_count[state] += 1
            prev_state = state
        if prev_state:
            emission_count[prev_state][word] = emission_count[prev_state].get(word, 0) + 1

    total_sentences = len(data)
    start_transition_prob = {state: count / total_sentences for state, count in start_transition_count.items()}
    transition_prob = {state: {state2: count2 / state_count[state] for state2, count2 in count.items()} for state, count in transition_count.items()}
    emission_prob = {state: {word: count / state_count[state] for word, count in state_emission_count.items()} for state, state_emission_count in emission_count.items()}

    return start_transition_prob, transition_prob, emission_prob

def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p.get(st, 0) * emit_p[st].get(obs[0], 0), "prev": None}

    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_trans_prob = max(V[t-1][prev_st]["prob"] * trans_p[prev_st].get(st, 0) for prev_st in states)
            for prev_st in states:
                if V[t-1][prev_st]["prob"] * trans_p[prev_st].get(st, 0) == max_trans_prob:
                    max_prob = max_trans_prob * emit_p[st].get(obs[t], 0)
                    V[t][st] = {"prob": max_prob, "prev": prev_st}
                    break

    opt = []
    max_prob = max(value["prob"] for value in V[-1].values())
    previous = None
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            opt.append(st)
            previous = st
            break

    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]

    return opt

states = {}
observations = {}

for sentence in train_data:
    for line in sentence:
        word, tag = line.strip().split()
        states[tag] = states.get(tag, 0) + 1
        if tag not in observations:
            observations[tag] = {}
        observations[tag][word] = observations[tag].get(word, 0) + 1

state_list = list(states.keys())
start_transition_prob, transition_prob, emission_prob = compute_probabilities(train_data, state_list)
display(start_transition_prob, transition_prob, emission_prob)
predicted_tags_viterbi = [viterbi([word.split()[0] for word in sentence], state_list, start_transition_prob, transition_prob, emission_prob) for sentence in dev_in_data]
display(predicted_tags_viterbi)

def extract_entities_from_tags(tags):
    entities = []
    entity = []
    for tag in tags:
        if tag.startswith("B-"):
            if entity:
                entities.append(tuple(entity))
                entity = []
            entity.append(tag)
        elif tag.startswith("I-"):
            entity.append(tag)
        else:
            if entity:
                entities.append(tuple(entity))
                entity = []
    if entity:
        entities.append(tuple(entity))
    return set(entities)

TP = 0
FP = 0
FN = 0

for pred, actual in zip(predicted_tags_viterbi, dev_tags_actual):
    predicted_entities = extract_entities_from_tags(pred)
    actual_entities = extract_entities_from_tags(actual)
    TP += len(predicted_entities.intersection(actual_entities))
    FP += len(predicted_entities - actual_entities)
    FN += len(actual_entities - predicted_entities)

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f_score = 2 * precision * recall / (precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("F-score:", f_score)

{'O': 0.9289176090468497,
 'B-positive': 0.052234787291330104,
 'B-negative': 0.014001077005923533,
 'B-neutral': 0.004846526655896607,
 'I-neutral': 0.0,
 'I-positive': 0.0,
 'I-negative': 0.0}

{'O': {'O': 0.8856896848630963,
  'B-positive': 0.03650766316514551,
  'B-negative': 0.012226623041157224,
  'B-neutral': 0.0021353538832443605,
  'I-neutral': 0.0,
  'I-positive': 0.0,
  'I-negative': 0.0},
 'B-positive': {'O': 0.871551724137931,
  'B-positive': 0.002586206896551724,
  'B-negative': 0.0,
  'B-neutral': 0.0008620689655172414,
  'I-neutral': 0.0,
  'I-positive': 0.11637931034482758,
  'I-negative': 0.0},
 'B-negative': {'O': 0.8110236220472441,
  'B-positive': 0.0,
  'B-negative': 0.0,
  'B-neutral': 0.0,
  'I-neutral': 0.0,
  'I-positive': 0.0,
  'I-negative': 0.1784776902887139},
 'B-neutral': {'O': 0.7916666666666666,
  'B-positive': 0.0,
  'B-negative': 0.0,
  'B-neutral': 0.0,
  'I-neutral': 0.20833333333333334,
  'I-positive': 0.0,
  'I-negative': 0.0},
 'I-neutral': {'O': 0.3488372093023256,
  'B-positive': 0.0,
  'B-negative': 0.0,
  'B-neutral': 0.0,
  'I-neutral': 0.6511627906976745,
  'I-positive': 0.0,
  'I-negative': 0.0},
 'I-positive': {'O': 0.42675159235

{'O': {'hace': 0.0008610297916307905,
  'poco': 0.0018598243499225074,
  'mi': 0.0024797657998966763,
  'pareja': 0.00044773549164801103,
  'y': 0.029929395557086274,
  'yo': 0.001171000516617875,
  'comiendo': 0.00030997072498708454,
  'resultó': 0.00010332357499569486,
  'todo': 0.0038918546581711727,
  'muy': 0.009609092474599621,
  'bien': 0.005476149474771827,
  ',': 0.050800757706216636,
  'tanto': 0.0012743240916135699,
  'la': 0.025452040640606164,
  'comida': 0.007783709316342345,
  'el': 0.021801274324091614,
  'vino': 0.0012398828999483382,
  'trato': 0.0020320303082486655,
  'decoración': 0.00027552953332185293,
  'nos': 0.004546237299810574,
  'gustó': 0.00037885310831754776,
  'mucho': 0.0018253831582572756,
  '.': 0.10556225245393491,
  'poner': 0.00017220595832615808,
  'algún': 0.0002066471499913897,
  'pero': 0.006440502841398312,
  'quizá': 0.00013776476666092647,
  'jamón': 0.00013776476666092647,
  'era': 0.0014465300499397278,
  'lo': 0.009058033407955916,
  '"': 

[['O', 'B-positive', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'B-positive', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  '

Precision: 0.7
Recall: 0.0748663101604278
F-score: 0.13526570048309178


In [43]:
import numpy as np

sr_start_probabilities = pd.Series(start_transition_prob)
df_transition = pd.DataFrame(transition_prob).T
# df_emission = pd.DataFrame(emission_prob).fillna(0).T
df_emission = create_e_x_y_df_train_all_y_values(file_path_train_es).pivot(index='y', columns='x', values='e(x|y)').fillna(0)

display(sr_start_probabilities, df_transition, df_emission)

display(df_emission.loc['O','!'])

unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}


O             0.928918
B-positive    0.052235
B-negative    0.014001
B-neutral     0.004847
I-neutral     0.000000
I-positive    0.000000
I-negative    0.000000
dtype: float64

,O,B-positive,B-negative,B-neutral,I-neutral,I-positive,I-negative
O,0.885690,0.036508,0.012227,0.002135,0.000000,0.000000,0.000000
B-positive,0.871552,0.002586,0.000000,0.000862,0.000000,0.116379,0.000000
B-negative,0.811024,0.000000,0.000000,0.000000,0.000000,0.000000,0.178478
B-neutral,0.791667,0.000000,0.000000,0.000000,0.208333,0.000000,0.000000
I-neutral,0.348837,0.000000,0.000000,0.000000,0.651163,0.000000,0.000000
I-positive,0.426752,0.000000,0.000000,0.000000,0.000000,0.570064,0.000000
I-negative,0.397661,0.000000,0.000000,0.000000,0.000000,0.000000,0.602339


x,!,"""",%,(,),):,);,*,+,",",...,últimos,única,único,‘,’,“,”,…,″,€
y,,,,,,,,,,,,,,,,,,,,,
B-negative,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.002625,0.000000,0.000000
B-neutral,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B-positive,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
I-negative,0.000000,0.011696,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
I-neutral,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
I-positive,0.000000,0.006369,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006369,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.003185,0.003185,0.000000,0.000000,0.003185
O,0.005442,0.001343,0.000413,0.004512,0.004512,0.000034,0.000034,0.000034,0.000069,0.057310,...,0.000034,0.000241,0.00031,0.000069,0.000276,0.000999,0.001171,0.001550,0.000034,0.000895


0.005441708283106595

In [41]:
def modified_viterbi(observation, transition, emission, start_probabilities, kth_best):

    states = transition.index.tolist()
    state_count = emission.shape[0]
    step_count = len(observation)
    
    # memoizing the incoming edges for each node in the viterbi trellis diagram
    df_incoming = np.zeros((state_count , step_count), dtype=int)
    df_incoming = pd.DataFrame(df_incoming, index = states)

    # memoizing the overall probabilities of arriving at each state at a particular step
    # START probabilities
    step_probabilities = emission[observation[0]] * start_probabilities
    step_probabilities_temp = pd.Series({state: 0 for state in states})

    for step in range(1, step_count):     
        for current_state in states:
            choosing_max = pd.Series({state: 0 for state in states})
            for previous_state in states:
                choosing_max.loc[previous_state] = step_probabilities.loc[previous_state] * transition.loc[previous_state, current_state] * emission.loc[current_state, observation[step]]
            max_state = choosing_max.idxmax()
            step_probabilities_temp.loc[current_state] = choosing_max.loc[max_state]
            df_incoming.loc[current_state, step] = max_state
        step_probabilities = step_probabilities_temp.copy()
    
    # take step_probabilities and multiply it by the STOP probabilities series
    # step_probabilities = step_probabilities * stop_probabilities

    # get sequences in descending order of likelihood
    descending_final_probability_states = step_probabilities.sort_values(ascending=False).index.to_numpy()
    df_sequences = np.zeros((state_count , step_count), dtype=int)
    df_sequences = pd.DataFrame(df_sequences)
    for sequence_rank in range(state_count):
        df_sequences.loc[sequence_rank, step_count-1] = descending_final_probability_states[sequence_rank]
    for sequence_rank in range(state_count):
        for step in range(step_count-2, -1, -1):
            previous_state = df_sequences.loc[sequence_rank, step+1]
            df_sequences.loc[sequence_rank, step] = df_incoming.loc[previous_state, step+1]

    return df_sequences.loc[kth_best-1].values

KeyError: 'posibilidades'

In [ ]:
predicted_tags_viterbi_2 = [modified_viterbi([word.split()[0] for word in sentence], df_transition, df_emission, sr_start_probabilities, 1) for sentence in dev_in_data]
predicted_tags_viterbi_8 = [modified_viterbi([word.split()[0] for word in sentence], df_transition, df_emission, sr_start_probabilities, 8) for sentence in dev_in_data]

TP = 0
FP = 0
FN = 0

for pred, actual in zip(predicted_tags_viterbi_2, dev_tags_actual):
    predicted_entities = extract_entities_from_tags(pred)
    actual_entities = extract_entities_from_tags(actual)
    TP += len(predicted_entities.intersection(actual_entities))
    FP += len(predicted_entities - actual_entities)
    FN += len(actual_entities - predicted_entities)

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f_score = 2 * precision * recall / (precision + recall)

print("stats for 2nd best")
print("Precision:", precision)
print("Recall:", recall)
print("F-score:", f_score)

for pred, actual in zip(predicted_tags_viterbi_8, dev_tags_actual):
    predicted_entities = extract_entities_from_tags(pred)
    actual_entities = extract_entities_from_tags(actual)
    TP += len(predicted_entities.intersection(actual_entities))
    FP += len(predicted_entities - actual_entities)
    FN += len(actual_entities - predicted_entities)

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f_score = 2 * precision * recall / (precision + recall)

print("stats for 8th best")
print("Precision:", precision)
print("Recall:", recall)
print("F-score:", f_score)